In [5]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
import numpy as np

#visualization libraries for graphing
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set_theme()


In [6]:
chemin_fichier = 'C:\\Users\\douja\\OneDrive\\Desktop\\Formation Data Analyst\\Projets Fil Rouge_Data\\Données Projet Financement Participatif//Kickstarter Campaigns DataSet.csv'

df = pd.read_csv(chemin_fichier)

# I- Features Processing

In [7]:
#le fichier que l'on va utiliser Kickstarter Campaigns DataSet.csv

df = df.drop(['Unnamed: 0','slug','name','blurb', 'creator_id','id'], axis = 1)
#on va compter les lignes en doubles
print("duplicates: ", df.duplicated().sum())
df.drop_duplicates(inplace=True, ignore_index=True)
# original row-length was 217245. check after removal of duplicates
print("length of dataset after removal of duplicates is ", len(df))

#supprimer les projets canceled et live du Dataset
df= df.loc[(df['status']=='successful') | (df['status']=='failed')]
print(df['status'].unique())


duplicates:  19527
length of dataset after removal of duplicates is  197718
['failed' 'successful']


In [8]:
import pandas as pd
from datetime import datetime


#
df['launched_at'] = pd.to_datetime(df['launched_at'], errors='coerce')

# Vérifier les valeurs de date invalides


# Extraire le jour de la semaine
df['day_of_week'] = df['launched_at'].dt.strftime('%A')

# Afficher le DataFrame modifié

df.head()

,currency,launched_at,backers_count,country,deadline,status,usd_pledged,sub_category,main_category,blurb_length,goal_usd,city,duration,day_of_week
0,CAD,2018-02-10 22:47:52,16,CA,2018-04-11 21:47:52,failed,577.902078,food,Farms,122,785.82065,Victoria,60.0,Saturday
1,USD,2014-11-21 10:42:26,7,US,2014-12-15 10:42:26,successful,211.000000,publishing,Academic,35,93.00000,San Diego,24.0,Friday
2,USD,2020-10-19 05:26:59,99,US,2020-11-18 05:26:59,successful,4697.000000,food,Small Batch,122,1500.00000,Philadelphia,30.0,Monday
3,USD,2019-05-29 16:01:34,3,US,2019-07-28 16:01:34,failed,222.000000,technology,Gadgets,69,25000.00000,Chicago,60.0,Wednesday
4,USD,2015-11-19 20:36:01,100,US,2015-12-19 20:36:01,successful,5410.000000,publishing,Radio & Podcasts,131,5000.00000,Madrid,30.0,Thursday


In [9]:
#Features Preprocessing
from datetime import datetime

df['launched_at'] = pd.to_datetime(df['launched_at'], format='%Y-%m-%d %H:%M')

#df['launched_at'] = df['launched_at'].apply(lambda df: pd.to_datetime(df['Date'], format='%Y-%m-%d %H:%M'))

import calendar
df['month'] = df['launched_at'].dt.month
df['year'] = df['launched_at'].dt.year
# Remplacer les mois numériques par les noms des mois
df['month'] = df['month'].apply(lambda x: calendar.month_name[x])
df.head(1)

,currency,launched_at,backers_count,country,deadline,status,usd_pledged,sub_category,main_category,blurb_length,goal_usd,city,duration,day_of_week,month,year
0,CAD,2018-02-10 22:47:52,16,CA,2018-04-11 21:47:52,failed,577.902078,food,Farms,122,785.82065,Victoria,60.0,Saturday,February,2018


In [10]:
data = df[['currency','country','city','sub_category','main_category','day_of_week', 'month', 'year','blurb_length','goal_usd','duration','status']]

data.head()

,currency,country,city,sub_category,main_category,day_of_week,month,year,blurb_length,goal_usd,duration,status
0,CAD,CA,Victoria,food,Farms,Saturday,February,2018,122,785.82065,60.0,failed
1,USD,US,San Diego,publishing,Academic,Friday,November,2014,35,93.00000,24.0,successful
2,USD,US,Philadelphia,food,Small Batch,Monday,October,2020,122,1500.00000,30.0,successful
3,USD,US,Chicago,technology,Gadgets,Wednesday,May,2019,69,25000.00000,60.0,failed
4,USD,US,Madrid,publishing,Radio & Podcasts,Thursday,November,2015,131,5000.00000,30.0,successful


In [11]:
# Encodage de la variable city
city_counts = data['city'].value_counts()
top_10_cities = city_counts.nlargest(10).index

data['city'] = data['city'].str.strip()

# Remplacer les noms de villes qui ne font pas partie des top 10 villes par "Other"
data['city'] = data['city'].apply(lambda x: x if x in top_10_cities else 'Other')

# Appliquer get_dummies() à la colonne 'city' modifiée
dummy_city_columns = pd.get_dummies(data['city'], prefix='city')

# Supprimer la colonne 'city_Other'
dummy_city_columns = dummy_city_columns.drop(['city_Other'], axis=1)

C:\Users\douja\AppData\Local\Temp\ipykernel_4656\131168419.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['city'] = data['city'].str.strip()
C:\Users\douja\AppData\Local\Temp\ipykernel_4656\131168419.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['city'] = data['city'].apply(lambda x: x if x in top_10_cities else 'Other')


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

#Identification des features et target

In [13]:
features = data.drop(['status'], axis = 1)
target = data['status']
features.head()
target

0             failed
1         successful
2         successful
3             failed
4         successful
             ...    
197712    successful
197713        failed
197714        failed
197716    successful
197717    successful
Name: status, Length: 184899, dtype: object

In [14]:
cat_feat = ['currency','country','sub_category','main_category','month','day_of_week']

cat_feat_dummies = pd.get_dummies(features[cat_feat])
cat_feat_dummies.head()

,currency_AUD,currency_CAD,currency_CHF,currency_DKK,currency_EUR,currency_GBP,currency_HKD,currency_JPY,currency_MXN,currency_NOK,...,month_November,month_October,month_September,day_of_week_Friday,day_of_week_Monday,day_of_week_Saturday,day_of_week_Sunday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0


In [15]:
feats  = pd.concat([cat_feat_dummies, dummy_city_columns], axis=1)
feats = pd.concat([features[[ 'goal_usd', 'duration', 'blurb_length','year']], feats], axis=1)
feats.head()

,goal_usd,duration,blurb_length,year,currency_AUD,currency_CAD,currency_CHF,currency_DKK,currency_EUR,currency_GBP,...,city_Atlanta,city_Brooklyn,city_Chicago,city_London,city_Los Angeles,city_New York,city_Portland,city_San Francisco,city_Seattle,city_Toronto
0,785.82065,60.0,122,2018,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,93.00000,24.0,35,2014,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1500.00000,30.0,122,2020,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,25000.00000,60.0,69,2019,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,5000.00000,30.0,131,2015,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
X_train, X_test,y_train, y_test = train_test_split(feats, target , test_size = 0.2, random_state =42 , stratify = target)


In [17]:
scaler = StandardScaler()

X_train[['goal_usd', 'duration', 'blurb_length', 'year']] = scaler.fit_transform(X_train[['goal_usd', 'duration', 'blurb_length', 'year']])
X_test[['goal_usd', 'duration', 'blurb_length', 'year']] = scaler.transform(X_test[['goal_usd', 'duration', 'blurb_length', 'year']])

X_train.head()

,goal_usd,duration,blurb_length,year,currency_AUD,currency_CAD,currency_CHF,currency_DKK,currency_EUR,currency_GBP,...,city_Atlanta,city_Brooklyn,city_Chicago,city_London,city_Los Angeles,city_New York,city_Portland,city_San Francisco,city_Seattle,city_Toronto
178942,-0.036008,2.223963,0.389319,0.261613,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
168034,-0.034547,-0.241326,0.793243,-0.556414,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
110309,-0.031415,-0.816559,-0.384870,0.261613,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
100975,-0.021472,2.223963,0.793243,-0.965427,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21252,-0.031415,-0.816559,0.759583,-2.192468,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
# Encodage de la variable categorielle


y_train = y_train.replace(to_replace=['successful', 'failed'], value = [1,0])

y_test = y_test.replace(to_replace=['successful', 'failed'], value = [1,0])

y_test.head()

43522     0
30160     1
136343    1
115724    1
146824    1
Name: status, dtype: int64

In [19]:
target.value_counts(normalize = True)


successful    0.59062
failed        0.40938
Name: status, dtype: float64

# II- Entrainement du modèl

In [20]:
import time

from sklearn.metrics import precision_score, recall_score, f1_score,accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler,  ClusterCentroids
from imblearn.metrics import classification_report_imbalanced, geometric_mean_score


In [21]:
  # Entraînez votre modèle ici
model = RandomForestClassifier(random_state = 42)
    
model.fit(X_train, y_train)
    # Prédisez les valeurs
y_pred = model.predict(X_test)
    


In [22]:
display(pd.crosstab(y_test,y_pred, rownames=['Realité'], colnames=['Prédiction']))

print('le rapport de classification du model Random Forest est \n',  classification_report(y_test, y_pred))

Prédiction,0,1
Realité,,
0,11965,3174
1,4029,17812


le rapport de classification du model Random Forest est 
               precision    recall  f1-score   support

           0       0.75      0.79      0.77     15139
           1       0.85      0.82      0.83     21841

    accuracy                           0.81     36980
   macro avg       0.80      0.80      0.80     36980
weighted avg       0.81      0.81      0.81     36980



# III- Over Sampling 

In [23]:
# OverSampler

rOs = RandomOverSampler()
X_ro, y_ro = rOs.fit_resample(X_train, y_train)
print('Classes échantillon oversampled :', dict(pd.Series(y_ro).value_counts()))

#SMOTE
smo = SMOTE()
X_sm, y_sm = smo.fit_resample(X_train, y_train)
print('Classes échantillon SMOTE :', dict(pd.Series(y_sm).value_counts()))


Classes échantillon oversampled : {1: 87364, 0: 87364}
Classes échantillon SMOTE : {1: 87364, 0: 87364}


In [24]:
  # Entraînez votre modèle ici 
model_Os = RandomForestClassifier(random_state = 42)
    
model_Os.fit(X_ro, y_ro)
    # Prédisez les valeurs
y_pred = model_Os.predict(X_test)
    

In [25]:
display(pd.crosstab(y_test,y_pred, rownames=['Realité'], colnames=['Prédiction']))

print('le rapport de classification imbalanced du model Random Forest est \n', classification_report_imbalanced(y_test, y_pred))
    

Prédiction,0,1
Realité,,
0,12517,2622
1,4581,17260


le rapport de classification imbalanced du model Random Forest est 
                    pre       rec       spe        f1       geo       iba       sup

          0       0.73      0.83      0.79      0.78      0.81      0.66     15139
          1       0.87      0.79      0.83      0.83      0.81      0.65     21841

avg / total       0.81      0.81      0.81      0.81      0.81      0.65     36980



In [26]:
accuracy = model_Os.score(X_test,y_test)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print('Accuracy :',    accuracy )
print('precision :',    precision )
print('recall:',   recall )
print('f1:',   f1)

Accuracy : 0.8052190373174689
precision : 0.8681219193240116
recall: 0.7902568563710453
f1: 0.827361407377226


In [27]:
model_sm = RandomForestClassifier(random_state = 42)
    
model_sm.fit(X_sm, y_sm)
    # Prédisez les valeurs
y_pred = model_sm.predict(X_test)

display(pd.crosstab(y_test,y_pred, rownames=['Realité'], colnames=['Prédiction']))

print('le rapport de classification imbalanced du model Random Forest est \n', classification_report_imbalanced(y_test, y_pred))
    

Prédiction,0,1
Realité,,
0,12748,2391
1,4767,17074


le rapport de classification imbalanced du model Random Forest est 
                    pre       rec       spe        f1       geo       iba       sup

          0       0.73      0.84      0.78      0.78      0.81      0.66     15139
          1       0.88      0.78      0.84      0.83      0.81      0.65     21841

avg / total       0.82      0.81      0.82      0.81      0.81      0.66     36980



In [28]:
accuracy = model_sm.score(X_test,y_test)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print('Accuracy :',    accuracy )
print('precision :',    precision )
print('recall:',   recall )
print('f1:',   f1)

Accuracy : 0.8064359113034072
precision : 0.8771641407654764
recall: 0.7817407627855868
f1: 0.8267079843122064


# IV- UnderSampling

In [29]:
#Random Undersampling
rUs = RandomUnderSampler()
X_ru, y_ru = rUs.fit_resample(X_train, y_train)
print('Classes échantillon undersampled :', dict(pd.Series(y_ru).value_counts()))



Classes échantillon undersampled : {0: 60555, 1: 60555}


In [30]:
  # Entraînez votre modèle ici 
model_ru = RandomForestClassifier(random_state = 42)
    
model_ru.fit(X_ru, y_ru)
    # Prédisez les valeurs
y_pred = model_ru.predict(X_test)

display(pd.crosstab(y_test,y_pred, rownames=['Realité'], colnames=['Prédiction']))

print('le rapport de classification imbalanced du model Random Forest est \n', classification_report_imbalanced(y_test, y_pred))
    

Prédiction,0,1
Realité,,
0,13371,1768
1,5660,16181


le rapport de classification imbalanced du model Random Forest est 
                    pre       rec       spe        f1       geo       iba       sup

          0       0.70      0.88      0.74      0.78      0.81      0.66     15139
          1       0.90      0.74      0.88      0.81      0.81      0.65     21841

avg / total       0.82      0.80      0.82      0.80      0.81      0.65     36980



In [31]:
accuracy = model_ru.score(X_test,y_test)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print('Accuracy :',    accuracy )
print('precision :',    precision )
print('recall:',   recall )
print('f1:',   f1)

Accuracy : 0.7991346673877772
precision : 0.9014986907348599
recall: 0.7408543564855089
f1: 0.8133199296305603
